<a href="https://colab.research.google.com/github/shivammehta007/Information-Retrieval/blob/master/Boolean_Retrieval_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boolean Retrieval Model

## Library Imports


In [85]:
# !pip install pymorphy2
import os
import gzip
import nltk
import pickle
# import pymorphy2

from nltk.corpus import stopwords
nltk.download("stopwords")

from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Reading Files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
indexing_data_location = '/content/drive/My Drive/Homeworks/InformationRetrieval/hw1_docs.tsv.gz'

In [4]:
with gzip.open(indexing_data_location, "rb") as f:
    print(f.readline().decode())
    print(f.readline().decode())
    print(f.readline().decode())
    print(f.readline().decode())
    print(f.readline().decode())
    line = f.readline().decode().split('\t')
    print(len(line), line)



7	PRESS RELEASES EMBASSY OF THE REPUBLIC OF UZBEKISTAN	OFFICE HOURS SATURDAY THURSDAY 8 AM 5 PM CONSULAR SECTION TUESDAY THURSDAY AND SUNDAY 9 AM 1 PM EMERGENCY ASSISTANCE ADDRESS STREET 3 COMMUNITY 314 UMM HURAIR FIRST BUR DUBAI AREA DUBAI UAE MAKANI NUMBER 29820 94373 TEL 9714 370 60 60 FAX 9714 370 60 63 E MAIL UZBEKCONSULATE AE MAIL RU P O BOX 50478 LOCATION MAP CONSULATE GENERAL OF THE REPUBLIC OF UZBEKISTAN IN DUBAI UAE PROFILE OF UZBEKISTAN STATE SYMBOLS CONSTITUTION PRESIDENT HISTORY PEOPLE CULTURE ECOLOGY NATIONAL HOLIDAYS ECONOMY DEVELOPMENT OF INTERNATIONAL RELATIONS TRAVELING TO UZBEKISTAN INVITATION TO UZBEKISTAN TOURISM IN UZBEKISTAN HISTORICAL PLACES TOUR COMPANIES HOTELS IN UZBEKISTAN BUSINESS IN UZBEKISTAN WHY UZBEKISTAN INVESTMENT GUIDE INVESTMENT OPPORTUNITIES INVESTMENT PROJECTS MULTIMEDIA S VIDEO AUDIO PHOTO HOME CONSULAR SERVICE O ZBEKCHA НА РУССКОМ IN ENGLISH BILETARIAL RELATION PRESS RELEASES EVENTS CONTACT US USEFUL LINKS 25 TH ANNIVERSARY OF INDEPENDENCE OF UZ

In [68]:
test_sen = 'OFFICE HOURS SATURDAY THURSDAY 8 AM 5 PM CONSULAR SECTION TUESDAY THURSDAY AND SUNDAY 9 AM 1 PM EMERGENCY ASSISTANCE ADDRESS STREET 3 COMMUNITY 314 UMM HURAIR FIRST BUR DUBAI AREA DUBAI UAE MAKANI NUMBER 29820 94373 TEL 9714 370 60 60 FAX 9714 370 60 63 E MAIL UZBEKCONSULATE AE MAIL RU P O BOX 50478 LOCATION MAP CONSULATE GENERAL OF THE REPUBLIC OF UZBEKISTAN IN DUBAI UAE PROFILE OF UZBEKISTAN STATE SYMBOLS CONSTITUTION PRESIDENT HISTORY PEOPLE CULTURE ECOLOGY NATIONAL HOLIDAYS ECONOMY DEVELOPMENT OF INTERNATIONAL RELATIONS TRAVELING TO UZBEKISTAN INVITATION TO UZBEKISTAN TOURISM IN UZBEKISTAN HISTORICAL PLACES TOUR COMPANIES HOTELS IN UZBEKISTAN BUSINESS IN UZBEKISTAN WHY UZBEKISTAN INVESTMENT GUIDE INVESTMENT OPPORTUNITIES INVESTMENT PROJECTS MULTIMEDIA S VIDEO AUDIO PHOTO HOME CONSULAR SERVICE O ZBEKCHA НА РУССКОМ IN ENGLISH BILETARIAL RELATION PRESS RELEASES EVENTS CONTACT US USEFUL LINKS 25 TH ANNIVERSARY OF INDEPENDENCE OF UZBEKISTAN ЛЕГКАЯ ПРОМЫШЛЕННОСТЬ УЗБЕКИСТАНА УКРЕПЛЯТЬ И РАСШИРЯТЬ ПОЗИЦИИ НА МИРОВОМ РЫНКЕ 7 МИЛЛИОНОВ ДОЛЛАРОВ И БОЛЕЕ 1 МИЛЛИАРДА ДОЛЛАРОВ СРАВНИТЕ ЭТИ ЦИФРЫ ПОЧУВСТВОВАЛИ РАЗНИЦУ А ТАК КАК ПЕРВАЯ ЦИФРА ПОКАЗЫВАЕТ ОБЪЕМ ЭКСПОРТА ТЕКСТИЛЬНОЙ И ШВЕЙНО ТРИКОТАЖНОЙ ПРОДУКЦИИ ОТЕЧЕСТВЕННЫХ ПРЕДПРИЯТИЙ В 1994 ГОДУ А ВТОРАЯ ХАРАКТЕРИЗУЕТ ТОТ ЖЕ ПОКАЗАТЕЛЬ НО УЖЕ ЗА 2015 ГОД ТО МОЖНО С ПОЛНОЙ УВЕРЕННОСТЬЮ СКАЗАТЬ ЧТО ЗА ГОДЫ НЕЗАВИСИМОСТИ ЛЕГКАЯ ПРОМЫШЛЕННОСТЬ УЗБЕКИСТАНА НЕ ТОЛЬКО УВЕРЕННО ВЫШЛА НА ВНЕШНИЙ РЫНОК НО И ЗАНЯЛА НА НЕМ ПРОЧНЫЕ ПОЗИЦИИ ПРОДОЛЖАЯ АКТИВНО НАРАЩИВАТЬ ЭКСПОРТНЫЙ ПОТЕНЦИАЛ ЧТОБЫ ЛУЧШЕ ПОНЯТЬ И ПО ДОСТОИНСТВУ ОЦЕНИТЬ ТУ ГРАНДИОЗНУЮ РАБОТУ КОТОРАЯ БЫЛА ПРОВЕДЕНА РУКОВОДСТВОМ РЕСПУБЛИКИ И ТРУЖЕНИКАМИ ОТРАСЛИ В ДАННОМ НАПРАВЛЕНИИ НАДО ВСПОМНИТЬ ПОЛОЖЕНИЕ СУЩЕСТВОВАВШЕЕ В ЛЕГКОЙ ПРОМЫШЛЕННОСТИ УЗБЕКИСТАНА В НАЧАЛЕ 90 Х ГОДОВ ПРОШЛОГО ВЕКА КАК ТО В ТЕ ВРЕМЕНА ДОВЕЛОСЬ ПРИСУТСТВОВАТЬ НА ВСТРЕЧЕ НАШИХ СПЕЦИАЛИСТОВ С ЗАРУБЕЖНЫМИ КОЛЛЕГАМИ ПРИЕХАВШИМИ В НАШУ СТРАНУ И ИНТЕРЕСОВАВШИМИСЯ КАК РАЗ СИТУАЦИЕЙ В РЯДЕ ОТРАСЛЕЙ В ТОМ ЧИСЛЕ В ЛЕГКОЙ ПРОМЫШЛЕННОСТИ А ТОЧНЕЕ ПРОИЗВОДСТВОМ ТЕКСТИЛЬНОЙ ПРОДУКЦИИ ОНИ НИКАК НЕ МОГЛИ ПОНЯТЬ ПОЧЕМУ В УЗБЕКИСТАНЕ ГДЕ ВЫРАЩИВАЕТСЯ СТОЛЬКО ХЛОПКА ПРАКТИЧЕСКИ НЕТ СОБСТВЕННЫХ ТКАНЕЙ ГОТОВОЙ ОДЕЖДЫ А НА РЫНКАХ ЛЮДЯМ ПРЕДЛАГАЮТ ИМПОРТНЫЙ ТОВАР ПРИЧЕМ СОМНИТЕЛЬНОГО КАЧЕСТВА ЭТОТ ВОПРОС ПОЧЕМУ ЧАЩЕ ВСЕГО ЗВУЧАЛ В ЗАЛЕ НАПРИМЕР ПОЧЕМУ В РЕСПУБЛИКЕ ЗАНИМАЮТСЯ В ОСНОВНОМ ТОЛЬКО ПОЛУЧЕНИЕМ ВОЛОКНА ИЗ ХЛОПКА СЫРЦА ИЛИ ПОЧЕМУ ИЗ ОБЩЕГО КОЛИЧЕСТВА ВЫРАБОТАННОГО ВОЛОКНА В ПРЯЖУ И ТКАНИ ПЕРЕРАБАТЫВАЕТСЯ ВСЕГО ЛИШЬ 10 ПРОЦЕНТОВ ПОЧЕМУ НА ЭКСПОРТ ОТПРАВЛЯЕТСЯ ТОЛЬКО ХЛОПКОВОЕ ВОЛОКНО А НЕ ГОТОВЫЕ ИЗДЕЛИЯ С БОЛЕЕ ВЫСОКОЙ ДОБАВЛЕННОЙ СТОИМОСТЬЮ ЧТО ГОРАЗДО ПРИБЫЛЬНЕЕ ПОЧЕМУ ОТПРАВЛЯЯ ВЫРАЩЕННЫЙ НЕЛЕГКИМ ТРУДОМ ХЛОПОК ЗА ПРЕДЕЛЫ РЕСПУБЛИКИ ТОТ ЖЕ ХЛОПКОРОБ ЗАТЕМ ВЫНУЖДЕН ПОКУПАТЬ РУБАШКУ ИЗ УЗБЕКСКОГО ХЛОПКА ЗА ГОРАЗДО БОЛЬШУЮ ЦЕНУ И ПОДОБНЫХ ВОПРОСОВ БЫЛО НЕМАЛО НАДО ОТДАТЬ ДОЛЖНОЕ НАШИМ СПЕЦИАЛИСТАМ ОНИ ПЫТАЛИСЬ ОТВЕТИТЬ НА ВСЕ ВОПРОСЫ ГОВОРИЛИ О ТОМ ЧТО В БЫВШЕМ СОЮЗЕ УЗБЕКИСТАНУ ОТВОДИЛАСЬ РОЛЬ ОСНОВНОГО ПРОИЗВОДИТЕЛЯ ХЛОПКА СЫРЦА И ПОСТАВЩИКА ХЛОПКОВОГО ВОЛОКНА ДЛЯ ДАЛЬНЕЙШЕЙ ПЕРЕРАБОТКИ В ДРУГИЕ РЕСПУБЛИКИ О ТОМ ЧТО ОБОРУДОВАНИЕ ИСПОЛЬЗУЕМОЕ НА НЕМНОГОЧИСЛЕННЫХ ТЕКСТИЛЬНЫХ ПРЕДПРИЯТИЯХ БЫЛО МОРАЛЬНО УСТАРЕВШИМ И ФИЗИЧЕСКИ ИЗНОШЕННЫМ ЧТО НЕ ПОЗВОЛЯЛО ВНЕДРЯТЬ НОВЫЕ ТЕХНОЛОГИИ И ВЫПУСКАТЬ КОНКУРЕНТОСПОСОБНУЮ ПРОДУКЦИЮ О ТОМ ЧТО ВОПРОСАМИ ЭКСПОРТА ЗАНИМАЛСЯ ЦЕНТР КОТОРЫЙ И ОПРЕДЕЛЯЛ ЧТО СКОЛЬКО И В КАКИЕ СТРАНЫ ОТПРАВЛЯТЬ А САМА РЕСПУБЛИКА НАПРЯМУЮ ОТ ЭКСПОРТА ХЛОПКОВОГО ВОЛОКНА ДОХОДОВ НЕ ПОЛУЧАЛА МОЖНО БЫЛО ПОСОЧУВСТВОВАТЬ СПЕЦИАЛИСТАМ В ЧАСТНОСТИ ИМ ХОРОШО БЫЛО ИЗВЕСТНО ЧТО РАСПОЛАГАЯ ХОРОШЕЙ СЫРЬЕВОЙ БАЗОЙ ВЫГОДНО РАЗВИВАТЬ ТЕКСТИЛЬНОЕ ПРОИЗВОДСТВО ЧТО НА ПРОИЗВОДСТВО ПРЯЖИ И ТКАНЕЙ ПРИХОДИТСЯ ОКОЛО 80 ПРОЦЕНТОВ ДОБАВЛЕННОЙ СТОИМОСТИ КОНЕЧНОЙ ТОВАРНОЙ ПРОДУКЦИИ ТОГДА КАК НА ПРОИЗВОДСТВО ХЛОПКА ВОЛОКНА ОКОЛО 10 ПРОЦЕНТОВ ТО ЕСТЬ СТРАНЫ ЭКСПОРТЕРЫ ПРЯЖИ И ТКАНЕЙ ПОЛУЧАЮТ ДОХОДЫ В ВОСЕМЬ РАЗ ПРЕВЫШАЮЩИЕ ДОХОД ОТ ЭКСПОРТА ХЛОПКОВОГО ВОЛОКНА А ЕСЛИ ЭКСПОРТИРОВАТЬ ГОТОВУЮ ОДЕЖДУ ТО ДОХОДЫ ВЫРАСТУТ ЕЩЕ БОЛЬШЕ НО В ТЕ ВРЕМЕНА И В ТЕХ УСЛОВИЯХ ОПРЕДЕЛЕННЫХ БЫВШЕЙ КОМАНДНО АДМИНИСТРАТИВНОЙ СИСТЕМОЙ РЕШЕНИЯ ПРИНИМАЛИСЬ НЕ НА МЕСТАХ В СОЮЗНЫХ РЕСПУБЛИКАХ А В ТАК НАЗЫВАЕМОМ ЦЕНТРЕ СИТУАЦИЯ КАРДИНАЛЬНО ИЗМЕНИЛАСЬ ПОСЛЕ ОБРЕТЕНИЯ УЗБЕКИСТАНОМ НЕЗАВИСИМОСТИ РАЗРАБОТКИ И НАЧАЛА РЕАЛИЗАЦИИ ЦЕЛЕНАПРАВЛЕННОЙ СИСТЕМНОЙ ГОСУДАРСТВЕННОЙ ПРОМЫШЛЕННОЙ ПОЛИТИКИ НАЦЕЛЕННОЙ НА ДИВЕРСИФИКАЦИЮ И МОДЕРНИЗАЦИЮ ПОВЫШЕНИЕ КОНКУРЕНТОСПОСОБНОСТИ ОТЕЧЕСТВЕННОЙ ЭКОНОМИКИ ПОДДЕРЖКУ РАЗВИТИЯ ВЕДУЩИХ ОТРАСЛЕЙ В ТОМ ЧИСЛЕ ПО НАРАЩИВАНИЮ ИХ ЭКСПОРТНОГО ПОТЕНЦИАЛА ЗА СЧЕТ УГЛУБЛЕНИЯ ПЕРЕРАБОТКИ МЕСТНОГО СЫРЬЯ ПОСЛЕДОВАТЕЛЬНАЯ И ПЛАНОМЕРНАЯ ПЕРЕОРИЕНТАЦИЯ ЭКСПОРТА С СЫРЬЯ НА ГОТОВУЮ ПРОДУКЦИЮ С ВЫСОКОЙ ДОБАВЛЕННОЙ СТОИМОСТЬЮ ПОЗВОЛИЛА СФОРМИРОВАТЬ НОВУЮ СТРАТЕГИЮ РАЗВИТИЯ ЛЕГКОЙ ПРОМЫШЛЕННОСТИ НАЦЕЛЕННУЮ НА ПЕРСПЕКТИВУ МОЩНЫЙ ИМПУЛЬС НОВОМУ ЭТАПУ РАЗВИТИЯ ОТРАСЛИ ПРИДАЛИ ПРИНЯТЫЕ В РАЗНЫЕ ГОДЫ РУКОВОДСТВОМ РЕСПУБЛИКИ ПРОГРАММНЫЕ'
stemmer_en = nltk.SnowballStemmer("english")
stemmer_ru = nltk.SnowballStemmer("russian")

russian_stopwords = stopwords.words("russian")


# print([stemmer_ru.stem(word) for word in test_sen.split() if stemmer_ru.stem(word) not in stopwords.words("russian") ])
stemmed_sentence = [stemmer_ru.stem(word) for word in test_sen.split()]
tokens = [token for token in stemmed_sentence if token not in russian_stopwords\
            and token.strip() not in punctuation]
print(' '.join(tokens))


OFFICE HOURS SATURDAY THURSDAY 8 AM 5 PM CONSULAR SECTION TUESDAY THURSDAY AND SUNDAY 9 AM 1 PM EMERGENCY ASSISTANCE ADDRESS STREET 3 COMMUNITY 314 UMM HURAIR FIRST BUR DUBAI A DUBAI UAE MAKANI NUMBER 29820 94373 TEL 9714 370 60 60 FAX 9714 370 60 63 E MAIL UZBEKCONSULATE AE MAIL RU P O BOX 50478 LOCATION MAP CONSULATE GENERAL OF THE REPUBLIC OF UZBEKISTAN IN DUBAI UAE PROFILE OF UZBEKISTAN STATE SYMBOLS CONSTITUTION PRESIDENT HISTORY PEOPLE CULTURE ECOLOGY NATIONAL HOLIDAYS ECONOMY DEVELOPMENT OF INTERNATIONAL RELATIONS TRAVELING TO UZBEKISTAN INVITATION TO UZBEKISTAN TOURISM IN UZBEKISTAN HISTORICAL PLACES TOUR COMPANIES HOTELS IN UZBEKISTAN BUSINESS IN UZBEKISTAN WHY UZBEKISTAN INVESTMENT GUIDE INVESTMENT OPPORTUNITIES INVESTMENT PROJECTS MULTIME S VIDEO AUDIO PHOTO HOME CONSULAR SERVICE O ZBEK русск IN ENGLISH BILETARIAL RELATION PRESS RELEASES EVENTS CONTACT US USEFUL LINKS 25 TH ANNIVERSARY OF INDEPENDENCE OF UZBEKISTAN легк промышлен узбекиста укрепля расширя позиц миров рынк 7 

In [43]:
# Incase teacher allows lemmatizer
# morph = pymorphy2.MorphAnalyzer()

# for word in test_sen.split():
#     p = morph.parse(word)[0]
#     print(p.normal_form, end=" ")

office hours saturday thursday 8 am 5 pm consular section tuesday thursday and sunday 9 am 1 pm emergency assistance address street 3 community 314 umm hurair first bur dubai area dubai uae makani number 29820 94373 tel 9714 370 60 60 fax 9714 370 60 63 e mail uzbekconsulate ae mail ru p o box 50478 location map consulate general of the republic of uzbekistan in dubai uae profile of uzbekistan state symbols constitution president history people culture ecology national holidays economy development of international relations traveling to uzbekistan invitation to uzbekistan tourism in uzbekistan historical places tour companies hotels in uzbekistan business in uzbekistan why uzbekistan investment guide investment opportunities investment projects multimedia s video audio photo home consular service o zbekcha на русский in english biletarial relation press releases events contact us useful links 25 th anniversary of independence of uzbekistan лёгкий промышленность узбекистан укреплять и р

## Implementation of Inverted Index

In [0]:
class InvertedIndexer:
    """
    This class makes inverted index
    """

    def __init__(self, filename=False):
        self.filename = filename
        self.stemmer_ru = nltk.SnowballStemmer("russian")
        self.stopwords = stopwords.words("russian")
        self.punctuation = punctuation # from string import punctuation
        if filename:
            self.inverted_index = self._build_index(self.filename)
        else:
            self.inverted_index = {}

    def preprocess(self, sentence):
        """
        Method to remove stop words stem the sentence and return tokens
        """
        stemmed_sentence = [self.stemmer_ru.stem(word) for word in sentence.lower().split()]
        tokens = [token for token in stemmed_sentence if token not in self.stopwords\
                  and token.strip() not in self.punctuation]
    
        return tokens

    def _build_index(self, indexing_data_location):
        inverted_index = {}
        with gzip.open(indexing_data_location, "rb") as f:
            i = 0
            line = f.readline().decode().split('\t')
            while(line):
                if i ==5:
                    break
                file_number = line[0]
                subject = line[1]
                text = line[2]
                line = subject + ' ' + text
                
                for word in self.preprocess(line):
                    if inverted_index.get(word, None) is None:
                        inverted_index[word] = {file_number}
                    else:
                        inverted_index[word].add(file_number)
                line = f.readline().decode().split('\t')
                i += 1
            return inverted_index
    
    def save(self, filename_to_save):
        with open(filename_to_save, mode='wb') as f:
            pickle.dump(self.inverted_index, f)

    def load(self, filelocation_to_load):
        with open(filelocation_to_load, mode='rb') as f:
            self.inverted_index = pickle.load(f)


    

In [0]:
hw1 = InvertedIndexer(indexing_data_location)
hw1.inverted_index
hw1.save('inverted_index.p')

In [0]:
class SolutionPredictor:
    """
    This classes uses object of Hw1SolutionIndexer
    to make boolean search
    """
    def __init__(self, indexer):
        """
        indexer : object of class Hw1SolutionIndexer
        """
        self.indexer = indexer
        self.dictionary = indexer.inverted_index.keys()

    
    def find_docs(self, query):
        """
        This method provides booleaen search
        query : string with text of query
        Returns Python set with documents which contain query words
        Should return maximum 100 docs
        """
        # Your code is here
        tokens = self.indexer.preprocess(query)
        docs_list = None
        for word in tokens:
        if docs_list is None:
            docs_list = inverted_index.get(word)
        else:
            docs_list.intersection_update(inverted_index.get(word))
        return docs_list
